In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# translator pipeline for english to swahili translations
eng_swa_model_checkpoint = "Helsinki-NLP/opus-mt-en-swc"
eng_swa_tokenizer = AutoTokenizer.from_pretrained("../model/eng_swa_model/")
eng_swa_model = AutoModelForSeq2SeqLM.from_pretrained("../model/eng_swa_model/")

eng_swa_translator = pipeline(
    "text2text-generation",
    model=eng_swa_model,
    tokenizer=eng_swa_tokenizer,
)

def translate_text_eng_swa(text):
    translated_text = eng_swa_translator(text, max_length=128, num_beams=5)[0]['generated_text']
    return translated_text

# translator pipeline for swahili to english translations
swa_eng_model_checkpoint = "Helsinki-NLP/opus-mt-swc-en"
swa_eng_tokenizer = AutoTokenizer.from_pretrained("../model/swa_eng_model/")
swa_eng_model = AutoModelForSeq2SeqLM.from_pretrained("../model/swa_eng_model/")

swa_eng_translator = pipeline(
    "text2text-generation",
    model=swa_eng_model,
    tokenizer=swa_eng_tokenizer,
)

def translate_text_swa_eng(text):
    translated_text = swa_eng_translator(text, max_length=128, num_beams=5)[0]['generated_text']
    return translated_text


/home/rogendo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-17 19:40:38.096133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 19:40:38.096691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 19:40:38.243096: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 19:40:38.591501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is opt

In [1]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector


# Language detector
def get_lang_detector(nlp, name):
    return LanguageDetector()

# # Load the English language model
nlp = spacy.load("en_core_web_sm")

# # Register the language detection factory
Language.factory("language_detector", func=get_lang_detector)

# # Add the language detection component to the pipeline
nlp.add_pipe('language_detector', last=True)

# res="We went to the club for drinks. We were so drunk"


In [3]:

import csv
import os
import pandas as pd

def write_to_csv(user_input,translated_text, correct_translation):
    # Create a CSV file
    with open('../incorrect_translations/translations.csv', 'a', newline='') as file:
        # Create a CSV writer
        writer = csv.writer(file)

        # Check if the file is empty, i.e., it does not have a header row yet
        if os.path.getsize('translations.csv') == 0:
            # Write the column header to the file
            writer.writerow(['User Input', 'Translated Text', 'Correct Translation'])


        # Write the data to the CSV file
        writer.writerow([ user_input, translated_text, correct_translation])



# user_text="Once upon a time in the lush grasslands of East Africa, a wise old tortoise named Kiume lived a quiet, slow life. One sunny day, he stumbled upon a mischievous hare named Njeru. "
# user_text="Njeru, a master of deception, was known for tricking other animals into thinking he was a slower creature. This made him an outcast in the animal kingdom, and he often felt alone."
# user_text="Njeru eventually decided to join forces with Kiume. The two formed an unbreakable bond, with Njeru vowing to change his ways and start treating others with respect."
# user_text="Kiume, sensing Njeru's loneliness, offered his friendship. At first, Njeru hesitated, remembering the tricks he had played on others. But as time went by, he found himself growing fonder of Kiume."
user_text="You can access its rows, columns, and data by using standard indexing or by using column headers"
# language detector here

# language detection
doc=nlp(user_text)
detected_language = doc._.language['language']
print(f"Detected language: {detected_language}")   

if detected_language=="en":   
    translated_text=translate_text_eng_swa(user_text)
    print(translate_text_eng_swa(user_text))
elif detected_language=='sw':
    translated_text=translate_text_swa_eng(user_text)
    print(translate_text_swa_eng(user_text))





is_translation_correct = input("Is translation correct? ")
# is_translation_correct = bool(is_translation_correct)
if is_translation_correct.lower()=='yes': # If is_translation_correct is True
    print("correct translation")
    
elif  is_translation_correct.lower()=='no': # If is_translation_correct is False
    corrected_text = input("Enter the corrected text: ")
    
    print("correct translation " + corrected_text)
    write_to_csv(user_text, translated_text, corrected_text)



Detected language: en
Unaweza kupata safu zake, safu za nguzo, na habari kwa kuandika orodha ya kawaida au kwa kutumia safu za safu
correct translation


In [4]:
# # proprocess the saved csv file 
# def preprocess_csv(file):
#     import pandas as pd
#     df = pd.read_csv(file)
#     # remove rows with missing values (NaN or None)
#     df = df.dropna()
#     # df = df.
#     return df

# data = open('../incorrect_translations/translations.csv')
# preprocess_csv(data)    

In [5]:
import pandas as pd
df = pd.read_csv("../incorrect_translations/translations.csv")
# remove rows with missing values (NaN or None)
df = df.dropna()
df 


,User Input,Translated Text,Correct Translation
0,what was the original crime,Uhalifu wa awali ulisababishwa na nini?,uhalifu wa kwanza ulikuwa?
1,"Kiume, sensing Njeru's loneliness, offered his...","Kiume, ambaye alitambua upweke wa Njeru, alito...","Kiume ambaye alitambua upweke wa Njeru, alijit..."
2,"Kiume, sensing Njeru's loneliness, offered his...","Kiume, ambaye alitambua upweke wa Njeru, alito...","Kiume, ambaye alitambua upweke wa Njeru, aliji..."
3,Njeru eventually decided to join forces with K...,Mwishowe Njeru aliamua kujiunga na jeshi la Kiume,Mwishowe Njeru aliamua kujiunga na Kiume
4,Njeru eventually decided to join forces with K...,Mwishowe Njeru aliamua kujiunga na majeshi pam...,Mwishowe Njeru aliamua kujiunga pamoja na Kium...


In [6]:
# drop the 'Translated Text' column

df = df.drop('Translated Text', axis=1)


In [7]:
df.describe()

,User Input,Correct Translation
count,5,5
unique,4,5
top,"Kiume, sensing Njeru's loneliness, offered his...",uhalifu wa kwanza ulikuwa?
freq,2,1


In [8]:
df.columns

Index(['User Input', 'Correct Translation'], dtype='object')